# 1-D Framework
<p>This notebook represents the cumulative and current state of the 1D functions that I have yet created.</p>

In [27]:
import numpy as np
import matplotlib.pyplot as plt

class FlowModel:
    
    def __init__(self, length = 10, numElements = 10, conductivity = .1, diffusivity = .05, dispersivity = .1, timeDelta = 1, porosity = .25):
        self.length = length
        self.numElements = numElements
        self.conductivity = conductivity
        self.heads = np.zeros(numElements)
        self.scale = length/numElements
        self.plt = plt
        self.diffusivity = diffusivity
        self.dispersivity = dispersivity
        self.solutePeak = 0   # Accurate tracker for the solute peak
        self.concentrations = np.zeros(numElements)
        self.totalSolute = 0
        self.totalWater = 0
        self.timeDelta = timeDelta
        self.porosity = porosity
        self.pointConstants = []
        self.queueChanges = []
        
        
    # Flow Functions
    #---------------------------------------------------------------------------
    def flow(self, runs = 1, debug = False, trackChanges = False):
        '''
        Normal 1D flow equation
        '''
        if trackChanges:
            self.queueChanges = []
        
        for i in range(runs):
            for el in self.pointConstants:
                self.heads[el[0]] = el[1]
            queue = np.zeros(self.numElements)
            queue[:-1] += self.timeDelta*self.conductivity*(self.heads[1: ] - self.heads[:-1])/self.scale
            queue[1: ] += self.timeDelta*self.conductivity*(self.heads[:-1] - self.heads[1: ])/self.scale
            self.heads += queue
            
            if debug:
                print("Queue:")
                print(queue)
            
            if trackChanges:
                self.queueChanges.append(np.sum(np.abs(queue)))
            del queue

        if trackChanges:
            self.queueChanges = np.array(self.queueChanges)
        
    def averageVelocity(self):
        '''
        Determines the average velocity for each element of the array
        - Porosity is constant, so the bulk of this is just determining q
        - This should be 1 shorter than numElements
        '''
        q = np.zeros(self.numElements)
        q += self.conductivity*(self.heads[:-1] - self.heads[1: ])*(1/self.scale)
        return q/self.porosity
    

    # Solute Transport
    #---------------------------------------------------------------------------
    def advection(self):
        '''
        Models advection, the transport of solute with the bulk fluid motion
        - Follows the motion of the largest
        '''
        new = self.solutePeak + self.averageVelocity()
        if new > int(self.solutePeak):
            queue = np.zeros(self.numElements)   # Need to determine the direction of flow, because this flows to the right,
            queue[:-1] += (self.concentrations[:-1] - self.concentrations[1: ])  # but it isn't necessarily that direction.
            queue[1: ] += (self.concentrations[1: ] - self.concentrations[:-1])
            self.concentrations += queue
            del queue
            
    def forceAdvection(self):
        queue = np.copy(self.concentrations[:-1])
        self.concentrations[:-1] -= queue
        self.concentrations[1:] += queue
        del queue
    
    def diffusion(self, runs = 1):
        for i in range(runs):
            queue = self.concentrations*self.diffusivity*self.timeDelta
            self.concentrations -= 2*queue
            self.concentrations[:-1] += queue[1: ]
            self.concentrations[1: ] += queue[:-1]
    
    def forceDiffusion(self, runs = 1):
        for i in range(runs):
            queue = self.concentrations*self.diffusivity
            self.concentrations -= 2*queue
            self.concentrations[:-1] += queue[1: ]
            self.concentrations[1: ] += queue[:-1]

    def dispersion(self):
        dispersiveCoefficients = self.dispersivity*self.averageVelocity()
        
        
    # Plotting things
    #---------------------------------------------------------------------------
    def plotHeads(self):
        self.plt.plot(np.linspace(0,self.length,self.numElements),self.heads)
        self.plt.xlim(0,self.length)
        self.plt.xlabel("Length")
        self.plt.ylabel("Hydraulic head")
        self.plt.title("Hydraulic head over the model")
        self.plt.show()
        
    def plotConc(self):
        self.plt.plot(np.linspace(0,self.length,self.numElements),self.concentrations)
        self.plt.xlim(0,self.length)
        self.plt.xlabel("X position")
        self.plt.ylabel("Solute Concentration")
        self.plt.title("Solute Concentration in the model")
        self.plt.ylim(0,1)
        self.plt.xlim(0, self.length)
        self.plt.show()
        
    def showQueueChanges(self):
        '''Plots a graph of how the queue changes over some iterations'''
        self.plt.plot(self.queueChanges)
        self.plt.xlabel("Iteration")
        self.plt.ylabel("Total Queue")
        self.plt.title("Queue change over time")
        self.plt.show()
        
    def showQueueAcceleration(self):
        '''Show a graph of how the queue change changes over time over time'''
        acceleration = np.abs(self.queueChanges[1:]-self.queueChanges[:-1])
        self.plt.plot(acceleration)
        self.plt.xlabel("Iterations")
        self.plt.ylabel("Change in the queue change")
        self.plt.title("Change in the queue change over iterations")
        self.plt.show()
        
    # Determining some actual useful things
    #---------------------------------------------------------------------------
    def soluteEvolution(self, curves = 5, time = 10):
        '''
        Runs solute transport evolution.
        - Plots a given amount of graphs for solute concentration over a given time.
        - At the moment, it does not contain dispersion
        '''
        plots = []
        step = int(time/curves)
        j = step
        alpha_step = .8/curves
        for i in range(time):
            mod.diffusion()
            mod.forceAdvection()
            if j == step:
                plots.append(np.copy(self.concentrations))
                j = 0
            else:
                j += 1
        for i in range(len(plots)):
            self.plt.plot(np.linspace(0,self.length,self.numElements),plots[i], color = 'blue', alpha = .2 + i*alpha_step)
        self.plt.xlabel("Horizontal position")
        self.plt.ylabel("Solute concentration")
        self.plt.xlim(0,self.length)
        self.plt.ylim(0,1)
        self.plt.title("Solute concentration over time")
        self.plt.show()
        
    
    # Printing information and things
    #---------------------------------------------------------------------------     
    def __str__(self):
        print("Flow Model")
        print("--------------")
        print("Number of elements:",self.numElements)
        print("Model Length:",self.length, "meters")
        print("Scale", self.scale, ("meters per element"))
        print("Hydraulic conductivity:", self.conductivity)
        return "--------------"
    
    def checkSums(self):
        if self.totalSolute != self.getSoluteTotal():
            print('Something wrong with solute:')
            print("Expected: ", self.soluteTotal)
            print("Current: ", self.getSoluteTotal())
    
    
    # Some ease of use stuff here
    #---------------------------------------------------------------------------
    def setDefaults(self):
        self.heads[0] = 1
        self.heads[-1] = 0
        self.pointConstants.append((0,1))
        self.pointConstants.append((self.numElements-1,0))
        self.concentrations[int(self.length/2)] = 1
        self.totalSolute = np.sum(self.concentrations)
        self.totalWater = np.sum(self.heads)
        
    def getSoluteTotal(self):
        return np.sum(self.concentrations)
    
    def getWaterTotal(self):
        return np.sum(self.heads)

## Testing the model

In [ ]:
mod = FlowModel(numElements = 10)
mod.setDefaults()
mod.flow(1000,trackChanges = True)
mod.showQueueChanges()

In [14]:
print(mod.queueChanges)

[20.0, 380.0, 11220.0, 365180.0, 12470420.0, 437881980.0, 15658177620.0, 567147750780.0, 20738973932820.0, 763959853171580.0, 2.830688868453602e+16, 1.0538539025333444e+18, 3.938981755439479e+19, 1.4771820742094716e+21, 5.55546458263955e+22, 2.0944767508574863e+24, 7.91341421289688e+25, 2.9955329769057945e+27, 1.135830758818811e+29, 4.313250868144752e+30, 1.6401419762618673e+32, 6.244352172759639e+33, 2.3799833422729413e+35, 9.080270866459316e+36, 3.4675693289791885e+38, 1.3253193522358734e+40, 5.0693971279773404e+41, 1.940470477848111e+43, 7.432758575388848e+44, 2.848819625285219e+46, 1.0925304331441264e+48, 4.1921740697738053e+49, 1.6094122784019908e+51, 6.181643927931017e+52, 2.3754012001430946e+54, 9.131770205348463e+55, 3.511944519676281e+57, 1.3511559736416471e+59, 5.2001963733112516e+60, 2.0020842588328342e+62, 7.710558648365372e+63, 2.9704572937103e+65, 1.1446911868972633e+67, 4.412400830953616e+68, 1.7012871103656814e+70, 6.561318448638958e+71, 2.5311077142534644e+73, 9.766334